# How to work with your azimuthally binned data

You can use this notebook as inspiration to help you process your azimuthally binned data and quickly prepare plots

### Content:
[**Import data**](#import)  
[**Plot average diffraction pattern**](#plot_pxrd)  
[**Plot azimuthal cake**](#plot_cake)  
[**Plot azimuthal cake in polar coordinates**](#plot_cake_polar)  

In [ ]:
%matplotlib widget
import DanMAX as DM
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
print('Current visit directory:')
print(f"'{os.getcwd().split('/scripts')[0]}'")

#### Import data <a id='import'></a>

In [ ]:
# Insert path for the .h5 file - TIP: Use tap for auto-complete
#fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/scan-####.h5'
fname = DM.getLatestScan()
#fname = DM.findScan(####)
# get the azimuthally integrated filename from master file name
azint_fname = fname.replace('raw', 'process/azint_binned').split('.')[0]+'_pilatus_integrated.h5'


with h5py.File(azint_fname, 'r') as af:
    azint_data = af['I'][:]
    azint_angle = af['phi'][:]
    # Determine if the file is integrated in tth or q
    try:
        r_unit = af['2th'][:]
        tth = True
    except KeyError:
        r_unit = af['q'][:]
        tth = False
    try:
        bin_bounds = af['bin_bounds'][:]
    except KeyError:
        bin_bounds = None
azint_data[azint_data<=0]= np.nan

# estimate radial unit boundaries
r_bounds = r_unit[1:]-np.diff(r_unit)/2
r_bounds = np.insert(r_bounds,0,r_unit[0]-np.mean(np.diff(r_unit)/2))
r_bounds = np.append(r_bounds,r_unit[-1]+np.mean(np.diff(r_unit)/2))

# read common meta data from the master file
meta = DM.getMetaData(fname)
#meta_dic = DM.getMetaDic(fname) # extended meta data dictionary
t = meta['time'] # relative time stamp in seconds
T = meta['temp'] # temperature in Kelvin (if available, otherwise None)
I0 = meta['I0']  # relative incident beam intensity "I zero"
E = meta['energy'] # X-ray energy in keV


#### Plot average diffraction pattern <a id='plot_pxrd'></a>

In [ ]:
frame = 0
plt.figure()
plt.suptitle(f'Average diffraction pattern - Frame {frame:5d}')
plt.plot(r_unit, np.nanmean(azint_data[frame, :,:],axis=0))
plt.ylabel('I [a.u.]')
if tth:
    plt.xlabel(r'2$\theta$ [$\deg$]')
else:
    plt.xlabel(r'Q [$\AA^{-1}$]')

#### Plot azimuthal cake with variable azimuthal steps <a id='plot_cake'></a>  
Simpel framewise heatmap of the azimuthal cake. Allows non-equidistant azimuthal steps.

In [ ]:
frame = 0
# Generate and show the waterfall plot
plt.figure()
plt.suptitle(f'Azimuthal cake - Frame {frame:5d}')
if type(bin_bounds) == type(None):
    plt.pcolormesh(r_unit, azint_angle, azint_data[frame,:,:],
                   shading = 'Nearest',
                   norm='log')
else:
    plt.pcolormesh(r_bounds, bin_bounds, azint_data[frame,:,:],
                   shading = 'flat',
                   norm='log')
    plt.yticks(bin_bounds,minor=True)
plt.ylabel(r'Azimuthal angle [$\deg$]')
if tth:
    plt.xlabel(r'2$\theta$ [$\deg$]')
else:
    plt.xlabel(r'Q [$\AA^{-1}$]')

#### Plot azimuthal cake in polar coordinates <a id='plot_cake_polar'></a>  
Useful for visualizing the azimuthal bins

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.grid(False)
ax.pcolormesh(bin_bounds*np.pi/180, 
              r_bounds, 
              azint_data[frame,:,:].T,
              shading = 'auto',
              norm='log')

ax.set_theta_direction('clockwise')
ax.set_thetagrids(bin_bounds)
ax.set_rgrids([])
ax.grid(True)